In [2]:
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split


c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, ReLU, Flatten, LSTM, Input, MaxPooling1D, Lambda, MaxPooling2D
from tensorflow.keras.layers import Input, Embedding, BatchNormalization, Concatenate, Reshape
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Conv2D, SpatialDropout1D
from tensorflow.keras.models import Sequential, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras import regularizers
from keras.utils import to_categorical
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

Load dữ liệu cần train

In [4]:
df=pd.read_csv('Twitter_3/data_train.csv',encoding='ISO-8859-1')
df_test=pd.read_csv('Twitter_3/data_test.csv',encoding='ISO-8859-1')

Load tokenizers của BertTokenizerFast

In [6]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
token_lens = []

for txt in df['clean_text'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_len=np.max(token_lens)
token_lens_test = []

for txt in df_test['clean_text'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))
    
max_len=np.max(token_lens_test)



ROS over sampler giúp cho mô hình chạy ổn định hơn va chia tap train test de chay mo hinh

In [ ]:
ros=RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['text_clean']).reshape(-1, 1), np.array(df['Sentiment']).reshape(-1, 1))
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['clean_text', 'Sentiment'])
X = train_os['clean_text'].values
y = train_os['Sentiment'].values
seed=42
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=seed)
X_test = df_test['clean_text'].values
y_test = df_test['Sentiment'].values

Bien nhan thanh dang onehotcoding

In [ ]:
ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()
y_test = ohe.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()


Chuan bi xay dung mo hinh 

In [ ]:
MAX_LEN=128
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

bert_model = TFBertModel.from_pretrained('bert-base-uncased')

def create_model(bert_model, max_len=MAX_LEN):
    
    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    embeddings = bert_model([input_ids,attention_masks])[1]
    
    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)
    
    model.compile(opt, loss=loss, metrics=accuracy)

[train_input_ids,train_attention_masks]=tokenize(X_train)
[val_input_ids,val_attention_masks]=tokenize(X_valid)
[test_input_ids,test_attention_masks]=tokenize(X_test)

Thuc hien viec training mo hinh voi du lieu da co

In [ ]:
model = create_model(bert_model, MAX_LEN)
model.summary()
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=4, batch_size=32)

kiem tra mo hinh voi du lieu test

In [ ]:
result_bert = model.predict([test_input_ids,test_attention_masks])
y_pred_bert=np.argmax(result_bert,axis=1)
y_true_test=np.argmax(y_test,axis=1)
print('\tClassification Report for BERT:\n\n',classification_report(y_true_test,y_pred_bert, target_names=['Negative', 'Neutral', 'Positive']))
